In [6]:
import os
import streamlit as st
import PyPDF2
import re
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
from langchain.output_parsers import PydanticOutputParser
from langchain_core.caches import InMemoryCache
import langchain
from langchain_openai import ChatOpenAI
from langchain_core.globals import set_llm_cache


# Set OpenAI API key from environment variable
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [21]:
def extract_info_from_pdf_new(pdf_file):
    # Read PDF content
    reader = PyPDF2.PdfReader(pdf_file)
    resume_text = ""
    for page in reader.pages:
        resume_text += page.extract_text()

    openai_llm = ChatOpenAI(model="gpt-4o-mini", max_tokens=1000, temperature=0.7)

    prompt_template_resume = PromptTemplate(
        input_variables=["resume_text"],
        template="""
        Extract only the skills section from the given resume. If a skills section is not clearly defined, extract the most relevant information related to skills.
        Provide a maximum of 10 most relevant and important skills.

        Resume:
        {resume_text}

        Please provide the extracted information in the following format:

        Skills:
        [List of up to 10 skills, one per line]
        """,
    )

    # Set up LLMChain for the skills extraction
    extraction_chain = LLMChain(llm=openai_llm, prompt=prompt_template_resume)

    # Get the extracted skills by running the chain
    extracted_skills = extraction_chain.run({"resume_text": resume_text})

    # Extract the skills list, remove hyphens, and limit to 15 skills
    skills_section = extracted_skills.split("Skills:")[1].strip()
    skills_list = [skill.strip().lstrip('- ') for skill in skills_section.split("\n") if skill.strip()][:10]

    # If skills_list is empty, use a default list of skills
    if not skills_list:
        skills_list = ["C++", "Python", "Java"]

    return list(set(skills_list))

In [22]:
skill_list=extract_info_from_pdf_new("Ashish_Resume_ATS.pdf")

In [23]:
skill_list

['Python',
 'Generative AI',
 'MongoDB',
 'OpenAI',
 'FastAPI',
 'Docker',
 'API Integration',
 'AWS',
 'SQL',
 'Flask']

In [3]:
file_path = "./logical.txt"
def read_file(file_path):
    try:
        with open(file_path, "r") as file:
            data = file.read()  # Read the entire file
            # print ("data",data)
            return data
    except FileNotFoundError:
        print("File not found.")
        
def generate_questions(skills_with_scale):
    # Read logical reasoning questions from file
    logical_questions = read_file(file_path)
    if not logical_questions:
        print("Logical questions not loaded properly.")
        return

    print("Logical questions loaded successfully.")

    llm = ChatOpenAI(model="gpt-4o", max_tokens=4000, temperature=0.7)

    class Question(BaseModel):
        question: str = Field(description="The text of the quiz question")
        options: List[str] = Field(
            description="The multiple-choice options for the quiz question"
        )
        correct_answer: str = Field(
            description="The correct text answer for the quiz question"
        )

    class Category(BaseModel):
        category: str = Field(description="The category of the quiz questions")
        questions: List[Question] = Field(
            description="List of questions under the category"
        )

    class Quiz(BaseModel):
        quiz: List[Category] = Field(
            description="The list of quiz categories with questions"
        )

    parser = PydanticOutputParser(pydantic_object=Quiz)

    prompt_template = PromptTemplate(
        input_variables=[
            
            "skills",
            # "experience",
            # "projects",
            "logical_questions",
        ],
        template="""
        You are an expert in creating advanced educational content. Based on the following information, generate a challenging quiz with multiple-choice questions (MCQs) focused on the categories listed. Ensure each question has four answer options (labeled A, B, C, D) and only one correct answer.

        
        Skills with Scale: {skills}
        

        Categories and Number of Questions:
        1. Logical Reasoning Questions (10 questions):
           - Use the following pre-existing logical reasoning questions. Choose 10 questions randomly from the provided list:

           {logical_questions}

           - Do not modify the selected questions in any way. Use them exactly as provided, including options and correct answers.
           - Ensure that the chosen questions are diverse and cover different types of logical reasoning.
        
       
        2. Technical Skills Questions (20 questions):
           - Generate medium to advanced level questions based on the skills provided.
           - Distribute the 20 questions evenly among all the skills. If the number of skills doesn't divide evenly into 20, allocate extra questions to skills with higher scale values.
           - Create complex questions that require in-depth knowledge and application of the skills listed.
           - Focus on practical scenarios, problem-solving, and critical thinking relevant to the job role.
           - Include a mix of:
             a) Scenario-based questions that require analysis and decision-making
             b) Code snippet questions (where applicable) that test understanding of syntax and best practices
             c) Questions about advanced features or recent developments in the relevant technologies
             d) Problem-solving questions that require combining multiple concepts within a skill area
           - Ensure questions cover a range of topics from the provided skills, emphasizing those with higher skill scales.
           - Avoid overly basic or introductory-level questions.

        Instructions:
        - For Logical Reasoning Questions: Use the provided questions exactly as they are. Choose questions randomly, not sequentially.
        - For Technical Skills Questions: Create new, relevant questions based on the skills provided.
        - Ensure all questions are challenging and aligned with the advanced skill level required for the job position.
        - Strictly adhere to the specified number of questions for each category.
        - Each question must be directly relevant to the skills provided.
        - Design questions to test deep knowledge, critical thinking, and problem-solving abilities.
        - Provide a brief explanation for the correct answer after each technical skills question.

        Format the output as a JSON object with the structure defined by the following Pydantic models:

        {format_instructions}
    """,
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    # Create the LLM chain
    chain = LLMChain(llm=llm, prompt=prompt_template)

    # Generate the response
    try:
        response = chain.run(
            {
                
                "skills": skills_with_scale,
                # "experience": experience,
                # "projects": projects,
                "logical_questions": logical_questions,
            }
        )
        print("LLM response generated successfully.")

        # testing
        # test_case = LLMTestCase(input=prompt_template.template, actual_output=response)
        # relevancy_metric = AnswerRelevancyMetric(threshold=0.5)

        # relevancy_metric.measure(test_case)
        # print("relevancy_metric_score=",relevancy_metric.score)
        # print("relevancy_metric_reason=",relevancy_metric.reason)

    except Exception as e:
        print("Error during LLMChain execution:", str(e))
        return None

    # Parse the response into the structured format
    try:
        parsed_response = parser.parse(response)
        return parsed_response
    except Exception as e:
        print("Error parsing LLM response:", str(e))
        return None

In [4]:
skills={
    "c++": 8,
    "python": 9,
    "java": 7,
    "javascript": 6,
    "html": 5,
}

In [7]:
ans=generate_questions(skills)

Logical questions loaded successfully.


c:\Users\Acer\Desktop\Python-projects\Hidevs-projects\30_quiz\venv\Lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Users\Acer\Desktop\Python-projects\Hidevs-projects\30_quiz\venv\Lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


LLM response generated successfully.


In [8]:
ans

Quiz(quiz=[Category(category='Logical Reasoning', questions=[Question(question='Given a Series -10, -8, 6, 40, 102, ? Find what number would come in place of the question mark(?).', options=['105', '200', '216', '129'], correct_answer='200'), Question(question='25 % of a number is 8 less than one-third of that number. Find the number.', options=['92', '85', '96', '90'], correct_answer='96'), Question(question='A man standing near a railway track observes that a train passes him in 80 seconds but to pass by a 180 m long bridge, the same train takes 200 seconds. Find the speed of the train.', options=['2 m/s', '3 m/s', '1 m/s', '1.5 m/s'], correct_answer='1.5 m/s'), Question(question='A boatman can row a boat upstream at 14 km/hr and downstream at 20 km/hr. Find the speed of the boat in still water.', options=['17 km/h', '15 km/h', '18 km/h', '20 km/h'], correct_answer='17 km/h'), Question(question='Rohan said to Rashmi, â€˜ your motherâ€™s husbandâ€™s sister is my aunt.â€™ How is Rashmi